<a href="https://colab.research.google.com/github/PTson2207/Deployment-Model-App/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training someone model, then deploy ...

- Want: Save_model and then upload Google Storage
- Data: From Kaggle-Food 101
- Model(s): 10, 11, 12 classes

Setup some Function

In [1]:
# library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
# Function unzip dowload_data file
def unzip_data(file_name):
    zip_ref = zipfile.ZipFile(file_name, 'r')
    zip_ref.extractall()
    zip_ref.close()